# Pipelines Benchmarking :

- The methodology is so straightforward, first we should counstarct our grounding truth small dataset of extracted informations from Invoices, these can be done using GPT4 because is so good on that.
- Then we can compare each approach outputs with the grounding truth, and calculate **Accuaracy**.
- Three Benchmarks are Considered: `Accuracy`, `Inference time (s)` and `Cost`.
- The Grounding Truth dataset will be in a form of a csv file we will focus on Entities like :

    - TT,TTC,TVA,Date de facture,Numero facture, Supplier Name.

- All the Bechmarking process is done with a L4 machine.



In [1]:
project_path = "/content/drive/MyDrive/Textra_Insights/"

# Grounding Truth:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
# Path to your CSV file from drive
csv_file_path = project_path+"CSVs/grounding_truth.csv"
df = pd.read_csv(csv_file_path)
df.head()

,Invoice_Name,Fournisseur,Date_Facture,Num_Facture,TTC,TT,TVA
0,2.jpg,OUTIDIS SARL,13/01/2023,FV230007,586.80,489.00,97.80
1,3.jpg,FUN PLACE,14/04/2023,FUNPLACE - 2023/000403,3864.00,3220.00,644.00
2,49.jpg,LYDEC,30/03/2023,1507042573 AM,224.28,209.59,14.69
3,57.jpg,ARCANES TECHNOLOGIES,04-01-2023,223285-23-DJ,226.80,189.00,37.80
4,58.jpg,NETTOPAP,10/03/2023,230058,3026.45,2522.04,504.41


# Pipeline 1 :

## OCR

- A Blog by <a href="https://blog.roboflow.com/best-ocr-models-text-recognition/#ocr-testing-methodology">Roboflow</a> countain a good bechamrking of 4 OCR models (docTR, EasyOCR, Tesseract, Surya) Show that EasyOCR has the best accuarcy/speed ratio.
- Another comparaison between EasyOCR and <a href = "https://paddlepaddle.github.io/PaddleOCR/latest/en/index.html">PaddleOCR</a> by **Christian Weiler** <a href="https://www.bludelta.de/en/ocr-and-deepocr-in-comparison/">here</a> on 400 real invoices, show that EasyOCR with a GPU has the best accuracy/speed ratio.
- <a href="https://www.geeksforgeeks.org/introduction-to-levenshtein-distance/">Levenshtein distance</a>

Paddle Installation :
```bash
!pip install paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install "paddleocr>=2.0.1" # Recommend to use version 2.0.1+
```

EasyOCR Installation :
```bash
!pip install easyocr -q
```

In [2]:
!pip install paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install "paddleocr>=2.0.1" # Recommend to use version 2.0.1+

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.7/544.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=bfab9a6749f9d5fedec52c28fc310b7432fe22360c34f721e63e75f93e1f85ff
  Stored in directory

In [ ]:
!pip install easyocr -q

### Run OCR on All Invoices :

- PaddleOCR
- EasyOCR (Switch to T4 so We don't wait alot)

#### OCR_Functions

In [6]:
import time
import os
from tqdm import tqdm
import pandas as pd

################################################################################
################################### PaddleOCR ##################################
from paddleocr import PaddleOCR
paddle_reader = PaddleOCR(use_angle_cls=True, lang='fr')

def Run_Paddle(Image_path):
    Start = time.time()
    result = paddle_reader.ocr(Image_path, det=True, rec=True)
    End = time.time()
    # Inference Time
    Time = End-Start
    # Process the result
    extracted_text = ''
    #print("Lenght Result: ",len(result))
    for line in result:
        if line is not None:
            for word in line:
                #print("Word: ",word)
                # Access the text part of the tuple
                extracted_text += word[1][0] + ' '  # Access the first element of the recognized text (the text itself)
            extracted_text += '\n'
    return extracted_text,Time


def Run_Paddle_ALL(Folder_path):
    df = pd.DataFrame(columns=['InvoiceName', 'Text', 'Time(s)'])
    Files = os.listdir(Folder_path)
    Files = sorted(Files, key=lambda x: int(x.split('.')[0]))
    Texts = []
    Times = []
    for file in tqdm(Files,desc="Paddle Processing", unit="number"):
        print(f"{file} is processing")
        Invoice_path = os.path.join(Folder_path, file)
        Text_paddle,Time_paddle = Run_Paddle(Invoice_path)
        Texts.append(Text_paddle)
        Times.append(Time_paddle)
    df['InvoiceName'] = Files
    df['Text'] = Texts
    df['Time(s)'] = Times

    # Saving the DataFrame to a CSV file
    df.to_csv(project_path+'CSVs/PaddleOCR_Results.csv', index=False)

################################################################################
################################### EasyOCR ####################################

import easyocr
easyocr_reader = easyocr.Reader(['fr'],gpu = True)

def Run_EasyOCR(Image_path):
    Start = time.time()
    result = easyocr_reader.readtext(Image_path)
    End = time.time()

    # Inference Time
    Time = End-Start

    extracted_text = ''
    for line in result:
        extracted_text += line[1] + ' '
    return extracted_text,Time


def Run_EasyOCR_ALL(Folder_path):
    df = pd.DataFrame(columns=['InvoiceName', 'Text', 'Time(s)'])
    Files = os.listdir(Folder_path)
    Files = sorted(Files, key=lambda x: int(x.split('.')[0]))
    Texts = []
    Times = []
    for file in tqdm(Files,desc="EasyOCR Processing", unit="number"):
        print(f"{file} is processing")
        Invoice_path = os.path.join(Folder_path, file)
        Text_easyocr,Time_easyocr = Run_EasyOCR(Invoice_path)
        Texts.append(Text_easyocr)
        Times.append(Time_easyocr)
    df['InvoiceName'] = Files
    df['Text'] = Texts
    df['Time(s)'] = Times
    # Saving the DataFrame to a CSV file
    df.to_csv(project_path+'CSVs/EasyOCR_Results.csv', index=False)




download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:00<00:00, 4886.05it/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/latin_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer/latin_PP-OCRv3_rec_infer.tar


100%|██████████| 9930/9930 [00:01<00:00, 9592.20it/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:00<00:00, 2930.03it/s]


[2024/11/24 17:45:28] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25,

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

#### Test

In [7]:
Folder_path = project_path+"Test_Invoices"

In [ ]:
Run_Paddle_ALL(Folder_path)

In [8]:
Run_EasyOCR_ALL(Folder_path)

EasyOCR Processing:   0%|          | 0/30 [00:00<?, ?number/s]

2.jpg is processing


EasyOCR Processing:   3%|▎         | 1/30 [00:04<02:08,  4.42s/number]

3.jpg is processing


EasyOCR Processing:   7%|▋         | 2/30 [00:07<01:34,  3.38s/number]

...

EasyOCR Processing:  93%|█████████▎| 28/30 [01:48<00:07,  3.78s/number]

417.jpg is processing


EasyOCR Processing:  97%|█████████▋| 29/30 [01:50<00:03,  3.41s/number]

419.jpg is processing


EasyOCR Processing: 100%|██████████| 30/30 [01:53<00:00,  3.80s/number]


In [9]:
import pandas as pd
df_paddle = pd.read_csv(project_path+'CSVs/PaddleOCR_Results.csv')
df_paddle.head()

,InvoiceName,Text,Time(s)
0,2.jpg,FACTURE N°: FV230007 KiPRiNT Date Délai Livra...,3.732032
1,3.jpg,Facture N°:FUNPLACE -2023/000403 FUNART Date:1...,4.489319
2,49.jpg,Référence : 0026757005: Votre Direction Préfe...,5.463018
3,57.jpg,Page : 1/1 ARCANES TECHNOLOGIES FACTURE: 22328...,2.861523
4,58.jpg,NETTOOPAP PRODUITS DE NETTOYAGE ET D'HYGIENE S...,4.829792


In [10]:
df_easyocr = pd.read_csv(project_path+'CSVs/EasyOCR_Results.csv')
df_easyocr.head()

,InvoiceName,Text,Time(s)
0,2.jpg,FACTURE N' FV230007 KiPRINT Date Délai Livrais...,4.419701
1,3.jpg,"he Facture N"": FUNPLACE 2023/000403 FUN ART Da...",2.650285
2,49.jpg,Référence 0026 757 005 Votre Direction Préfect...,6.203648
3,57.jpg,ensombis Oisonsc Waroc(ehnoogguedrtaog Page 1 ...,2.691493
4,58.jpg,NETTO PAP PRODUITS DE NETTOYAGE ET D'HYGIÈNE S...,3.463183


## LLM

- From a quick search, you can find that LLM via API Providers like OpenAI, Anthropic... are the best in Information extraction.
But since we decide to keep everything localy, we need to test some of the local LLMs with Oallama and Huggingface.

- For All the tested models i use the q-4 quantized version of the model.

#### Install Dependencies

In [ ]:
!pip install langchain-openai==0.2.9

In [ ]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

In [24]:
%xterm
# curl https://ollama.ai/install.sh | sh
# ollama serve & ollama pull llama3.2:3b
# Then Type Ctrl+C
# ollama pull llama3.2:3b

Launching Xterm...

<IPython.core.display.Javascript object>

#### LLM_Functions

In [25]:
################################################################################
################################ Structered Output ################################
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class Extraction_Facture(BaseModel):
    """Les Informations principales à extraire a partir d'une facture française"""
    Nom_facture: str = Field("", description="Le nom et le format(.jpg, .png, .jpeg) de l'image de la facture")
    Fournisseur: str = Field("", description="Le nom du fournisseur de la facture")
    date_facture: str = Field("", description="La date de la facture")
    num_facture: str = Field("", description="L'identifiant de la facture, peut être numérique ou alphanumérique")
    ttc: float = Field(0.0, description="Toutes Taxes Comprises. Il s'agit du prix final total payé par le client. Il inclut le prix Total HT et la Total TVA.")
    tt: float = Field(0.0, description="Le total hors taxes de la facture")
    tva: float = Field(0.0, description="Le Total taxes")



system = """
Vous êtes un assistant puissant dans l'extraction d'informations a partir des factures francaises. \n
- Un OCR est appliqué sur l'image de la facture, lire le text et extraire les informations suivantes :  \n

- Fournisseur, Date_Facture, Num_Facture, TTC, TT, TVA \n
- Les 3 prix TT,TTC,TVA doivent être numérique sans indication de devise. \n
- le nom de Fournisseur doit être en majuscule. \n
"""

## Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Le text donné par OCR ici :\n {OCR_Output} \n le Nom de facture est : \n {Nom_facture} \n"),
    ]
)

################################################################################
################################ LLM ###########################################
from langchain_openai import ChatOpenAI
import pandas as pd
import time

LLM = ChatOpenAI(
    model="llama3.2:3b",
    #temperature=0,
    api_key='ollama',
    base_url="http://localhost:11434/v1",
)
LLM_Structered = LLM.with_structured_output(Extraction_Facture)
LLM_Extractor = prompt | LLM_Structered

def Run_LLM(OCR_Output, Nom_facture):
    Input = {"OCR_Output":OCR_Output, "Nom_facture":Nom_facture}
    response = LLM_Extractor.invoke(Input)
    return response

# The Chain can raise an exception if the LLM fails to respect the structured output, so we can fix a retry number
RETRIES = 3

def Run_LLM_ALL(OCR_dataframe,pipelineName):
    LLM_Results = pd.DataFrame(columns=['Temps(s)','Nom_facture', 'Fournisseur', 'Date_Facture', 'Num_Facture', 'TTC', 'TT', 'TVA'])
    for index, row in OCR_dataframe.iterrows():
        print(f"Processing {index+1}/{len(OCR_dataframe)}...")
        for i in range(RETRIES):
            try:
                strat_time = time.time()
                response = Run_LLM(row['Text'], row['InvoiceName'])
                end_time = time.time()
                response_time = end_time - strat_time
                LLM_Results.loc[index] = [response_time,row['InvoiceName'], response.Fournisseur, response.date_facture, response.num_facture, response.ttc, response.tt, response.tva]
                #print(LLM_Results)
                break
            except Exception as e:
                #print(f"Error: {e}")
                LLM_Results.loc[index] = [0.0,row['InvoiceName'], '', '', '', 0.0, 0.0, 0.0]
                print(f"Retrying {i+1}/{RETRIES}...\n")

    # Saving the DataFrame to a CSV file
    LLM_Results.to_csv(project_path+f'CSVs/{pipelineName}_Results.csv', index=False)







#### Load OCR Results

In [17]:
df_paddle = pd.read_csv(project_path+'CSVs/PaddleOCR_Results.csv')
df_paddle.head()

,InvoiceName,Text,Time(s)
0,2.jpg,FACTURE N°: FV230007 KiPRiNT Date Délai Livra...,3.732032
1,3.jpg,Facture N°:FUNPLACE -2023/000403 FUNART Date:1...,4.489319
2,49.jpg,Référence : 0026757005: Votre Direction Préfe...,5.463018
3,57.jpg,Page : 1/1 ARCANES TECHNOLOGIES FACTURE: 22328...,2.861523
4,58.jpg,NETTOOPAP PRODUITS DE NETTOYAGE ET D'HYGIENE S...,4.829792


In [18]:
df_easyocr = pd.read_csv(project_path+'CSVs/EasyOCR_Results.csv')
df_easyocr.head()

,InvoiceName,Text,Time(s)
0,2.jpg,FACTURE N' FV230007 KiPRINT Date Délai Livrais...,4.419701
1,3.jpg,"he Facture N"": FUNPLACE 2023/000403 FUN ART Da...",2.650285
2,49.jpg,Référence 0026 757 005 Votre Direction Préfect...,6.203648
3,57.jpg,ensombis Oisonsc Waroc(ehnoogguedrtaog Page 1 ...,2.691493
4,58.jpg,NETTO PAP PRODUITS DE NETTOYAGE ET D'HYGIÈNE S...,3.463183


In [27]:
index = 1
Text_paddle = df_paddle['Text'].values[index]
Text_easyocr = df_easyocr['Text'].values[index]
InvoiceName = df_paddle['InvoiceName'].values[index]

print("Text paddle :", Text_paddle)
print("Text easyocr :", Text_easyocr)
print("InvoiceName :", InvoiceName)

response = Run_LLM(Text_paddle, InvoiceName)
response

Text paddle : Facture N°:FUNPLACE -2023/000403 FUNART Date:14/04/2023 PLACE Client client:SMART CALLDISTRIBUTION ICE: 001685924000018 Prix Désignation Quantité Prix Total HT unitaire Atelier poterie 230.00 14 3,220.00 Une heure d'activitépoterie Notes/ Commentaires Montant HT 3,220.00 TVA 644.00 (20.00% Arrétée a Ia somme de TROIS MILLE HUIT CENT SOlXANTE-QUATRE DIRHAMS Total TTC 3,864.00 dh FUNPLACE 295,Angle BAhdetmonmen t RuPersee,e Etage NA6 Casablanca-R C: 392379 Fun Place 295,Angle Bd Abdelmoumen et rue Persée 1er étage-A6-Maarif 20100-Casablanca 20100 Maroc- +212660-380688/+212622-384880-IF:24999819-RC :392 379-CNSS:5792628-ICE: 002005684000080 

Text easyocr : he Facture N": FUNPLACE 2023/000403 FUN ART Date 14/04/2023 PLACE Client client: SMART CALL DISTRIBUTION ICE: 001685924000018 Désignation Prix Quantité Prix Total HT unitaire Atelier poterie Une heure daclivité poierie 230.00 14 3,220.00 Notes Commentaires Montant HT 3,220.00 TVA (20.00%) 644.00 Arrêtée à la somme de TROI

Extraction_Facture(Nom_facture='3.jpg', Fournisseur='FUNPLACE', date_facture='14/04/2023', num_facture='000403', ttc=3864.0, tt=0.0, tva=644.0)

#### Run LLM on All Invoices

In [28]:
pipelineName = "llama3.2_3b_Paddle"
Run_LLM_ALL(df_paddle,pipelineName)
#pipelineName = "Qwen2.5_7b_Easy"
#Run_LLM_ALL(df_easyocr,pipelineName)

Processing 1/30...
Processing 2/30...
Processing 3/30...
Processing 4/30...
Processing 5/30...
Processing 6/30...
Processing 7/30...
Processing 8/30...
Retrying 1/3...

Processing 9/30...
Processing 10/30...
Processing 11/30...
Processing 12/30...
Processing 13/30...
Processing 14/30...
Retrying 1/3...

Processing 15/30...
Retrying 1/3...

Processing 16/30...
Processing 17/30...
Processing 18/30...
Processing 19/30...
Processing 20/30...
Retrying 1/3...

Retrying 2/3...

Processing 21/30...
Processing 22/30...
Processing 23/30...
Retrying 1/3...

Retrying 2/3...

Processing 24/30...
Retrying 1/3...

Retrying 2/3...

Processing 25/30...
Processing 26/30...
Retrying 1/3...

Processing 27/30...
Processing 28/30...
Retrying 1/3...

Retrying 2/3...

Retrying 3/3...

Processing 29/30...
Processing 30/30...


#### Evaluation Function

In [30]:
!pip install python-Levenshtein


In [31]:
import Levenshtein
import pandas as pd

def Calculate_Similarity(string1, string2):
    if pd.isna(string1):
        return 0
    distance = Levenshtein.distance(string1, string2)
    score = 1-(distance/max(len(string1), len(string2)))
    return score

def Evaluate(LLM_Results, Ground_Truth):
    Total_Similarity = 0
    Total_Margin = 0
    Total_Time = 0
    for index, Ground_row in Ground_Truth.iterrows():
        print("######## Invoice Number : ", index)
        Line_Similarity = 0
        Line_margin = 0

        # Strings Similarities
        Line_Similarity = Calculate_Similarity(LLM_Results.loc[index,'Fournisseur'], Ground_row['Fournisseur']) + Calculate_Similarity(LLM_Results.loc[index,'Date_Facture'], Ground_row['Date_Facture']) + Calculate_Similarity(LLM_Results.loc[index,'Num_Facture'], Ground_row['Num_Facture'])
        Total_Similarity += Line_Similarity/3
        print("Total Similarity : ", Total_Similarity)

        # Prices Margins
        Line_margin = abs(LLM_Results.loc[index,'TTC'] - Ground_row['TTC']) + abs(LLM_Results.loc[index,'TT'] - Ground_row['TT']) + abs(LLM_Results.loc[index,'TVA'] - Ground_row['TVA'])
        Total_Margin += Line_margin
        print("Total Margin : ", Total_Margin)

        Total_Time += LLM_Results.loc[index,'Temps(s)']

    Accuarcy = Total_Similarity/len(Ground_Truth)
    Average_Time = Total_Time/len(Ground_Truth)
    Average_Margin = Total_Margin/len(Ground_Truth)
    return Accuarcy, Average_Time, Average_Margin


#### Evaluation of Performance

In [32]:
LLM_Results = pd.read_csv(project_path+ f'CSVs/{pipelineName}_Results.csv')
Ground_Truth = pd.read_csv(project_path+'CSVs/grounding_truth.csv')
Accuarcy, Average_Time, Average_Margin = Evaluate(LLM_Results, Ground_Truth)

######## Invoice Number :  0
Total Similarity :  0.6666666666666666
Total Margin :  189.0
######## Invoice Number :  1
Total Similarity :  1.387205387205387
Total Margin :  3114.0
######## Invoice Number :  2
Total Similarity :  2.3615643615643616
Total Margin :  3115.76
######## Invoice Number :  3
Total Similarity :  3.3615643615643616
Total Margin :  402926.76
######## Invoice Number :  4
Total Similarity :  3.377158903474693
Total Margin :  408979.66000000003
######## Invoice Number :  5
Total Similarity :  4.154936681252471
Total Margin :  408979.66000000003
######## Invoice Number :  6
Total Similarity :  4.945485851111985
Total Margin :  409814.34
######## Invoice Number :  7
Total Similarity :  5.755009660635794
Total Margin :  409814.34
######## Invoice Number :  8
Total Similarity :  6.558931229263245
Total Margin :  410314.34
######## Invoice Number :  9
Total Similarity :  7.558931229263245
Total Margin :  410434.34
######## Invoice Number :  10
Total Similarity :  8.356134

In [33]:
print("Accuarcy :(%) ", Accuarcy*100)
print("Average_Time(s) : ", Average_Time)
print("Average_Margin (DH) : ", Average_Margin)

Accuarcy :(%)  70.41694333893669
Average_Time(s) :  1.5611521244049071
Average_Margin (DH) :  16072.474666666667


In [41]:
#Run this Only for the fist time to define the dataframe
#Results= pd.DataFrame(columns=['PipelineName','Accuarcy (%)', 'Average_Time (s)', 'Average_Margin (DH)'])
#Results.to_csv(project_path+'CSVs/Results.csv', index=False)

In [42]:
Results = pd.read_csv(project_path+'CSVs/Results.csv')
Results

,PipelineName,Accuarcy (%),Average_Time (s),Average_Margin (DH)


In [43]:
Results.loc[len(Results)] = [pipelineName,Accuarcy*100, Average_Time, Average_Margin]
Results.to_csv(project_path+'CSVs/Results.csv', index=False)

In [44]:
Results

,PipelineName,Accuarcy (%),Average_Time (s),Average_Margin (DH)
0,llama3.2_3b_Paddle,70.416943,1.561152,16072.474667


# Pipeline 2 :

- In this pipeline we will use Vision language models (Multimodal LLM) to diracetly extract key entities from the invoice image.
- With Local Models the reulsts are so poor, the VLM can't even respect the system prompt, and perform alot of mistakes bigger models may perform better
- We worked here with Image encoding instead of Url because we will use Local VLMs.

## Convert Images to a base64 string

In [5]:
import base64
import os
import pandas as pd
from tqdm import tqdm

In [6]:
def Image_to_base64(image_path):
    try:
        with open(image_path, "rb") as image_file:
            image_binary = image_file.read()
            base64_encoded = base64.b64encode(image_binary)
            return base64_encoded.decode("utf-8")
    except FileNotFoundError:
        return "Error: File not found."
    except Exception as e:
        return f"Error: {e}"

def Encode_All(Folder_path):
    Files = os.listdir(Folder_path)
    Files = sorted(Files, key=lambda x: int(x.split('.')[0]))
    Encoding_df = pd.DataFrame(columns=['InvoiceName', 'Image_Base64'])
    for file in tqdm(Files,desc="Encoding", unit="number"):
        print(f"{file} is processing")
        Invoice_path = os.path.join(Folder_path, file)
        base64_encoded = Image_to_base64(Invoice_path)
        Encoding_df.loc[len(Encoding_df)] = [file, base64_encoded]
    # Saving the DataFrame to a CSV file
    Encoding_df.to_csv(project_path+'CSVs/Encoding.csv', index=False)

- Example :

In [ ]:
Image_path = project_path+"Test_Invoices/2.jpg"
encoding = Image_to_base64(Image_path)
encoding

In [ ]:
Encode_All(project_path+"Test_Invoices")

In [11]:
Encoding_df = pd.read_csv(project_path+'CSVs/Encoding.csv')
Encoding_df.head()

,InvoiceName,Image_Base64
0,2.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,3.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,49.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,57.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,58.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


## Benchamrking

#### Install Dependencies

In [ ]:
!pip install langchain-openai==0.2.9

In [ ]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

In [15]:
%xterm
# curl https://ollama.ai/install.sh | sh
# ollama serve & ollama pull llava-llama3:8b
# Then Type Ctrl+C
# ollama pull llava-llama3:8b

Launching Xterm...

<IPython.core.display.Javascript object>

#### VLM_Functions

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


system = """
Vous êtes un assistant puissant dans l'extraction d'informations a partir des factures francaises. \n

## Les Informations à extraire sont : \n

- Observer bien la facture et extraire les informations suivantes :  \n

   - Nom_facture : Un String, Le nom et le format(.jpg, .png, .jpeg) de l'image de la facture. \n
   - Fournisseur : Un String, Le nom du fournisseur de la facture. \n
   - date_facture : Un String, La date de la facture. \n
   - num_facture : Un String, L'identifiant de la facture, peut être numérique ou alphanumérique. \n
   - ttc : Un Float, Toutes Taxes Comprises. Il s'agit du prix final total payé par le client. Il inclut le prix Total HT et la Total TVA. \n
   - tt : Un Float, Le total hors taxes de la facture. \n
   - tva : Un Float, Le Total taxes. \n

## Structure de la réponse : \n

- Votre reponse doit être un dictionnaire python.

## Consignes : \n

- Les 3 prix TT,TTC,TVA doivent être numérique sans indication de devise. \n
- le nom de Fournisseur doit être en majuscule. \n
"""
## Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "user",
            [
                 {
                    "type": "text",
                    "text": system,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": "data:image/jpeg;base64,{image_data}"},
                }
            ],
        ),
    ]
)

VLM = ChatOpenAI(
    model="llava-llama3:8b",
    #temperature=0,
    api_key='ollama',
    base_url="http://localhost:11434/v1",
)

VLM_Extractor = prompt | VLM

def Run_VLM(InvoiceName,image_data):
    response = VLM_Extractor.invoke({"image_data": image_data,"InvoiceName": InvoiceName})
    return response

#### Test

In [18]:
encoding = Encoding_df.loc[0].Image_Base64
Name = Encoding_df.loc[0].InvoiceName
response = Run_VLM(Name,encoding)
response

AIMessage(content="The image presents a facture from Kiprint, a company that specializes in printing solutions for businesses. The facture is printed on white paper and is slightly tilted to the right. It's dated February 24, 2014, and issued number 14/0145.\n\nThe text on the document reveals that it's a facture from Kiprint, with the total price amounting to 400 euros. However, the actual body of the facture is not visible in the image, so specific details about the items purchased or their quantities cannot be determined.\n\nThe company's logo and contact information are also present on the document. The logo is a vibrant mix of colors - red, blue, green, yellow, orange, pink, purple, and black. The contact information includes a phone number labeled 'Tél.' and an email address labeled 'Email'. \n\nOverall, the image captures a moment in Kiprint's business operations, providing a glimpse into their facture issuance process.", additional_kwargs={'refusal': None}, response_metadata={'

In [19]:
print(response.content)

The image presents a facture from Kiprint, a company that specializes in printing solutions for businesses. The facture is printed on white paper and is slightly tilted to the right. It's dated February 24, 2014, and issued number 14/0145.

The text on the document reveals that it's a facture from Kiprint, with the total price amounting to 400 euros. However, the actual body of the facture is not visible in the image, so specific details about the items purchased or their quantities cannot be determined.

The company's logo and contact information are also present on the document. The logo is a vibrant mix of colors - red, blue, green, yellow, orange, pink, purple, and black. The contact information includes a phone number labeled 'Tél.' and an email address labeled 'Email'. 

Overall, the image captures a moment in Kiprint's business operations, providing a glimpse into their facture issuance process.


# Pipeline 3 :

## Parser

#### Parser_Functions

In [ ]:
!pip install docling

In [3]:
from paddleocr import PaddleOCR
paddle_reader = PaddleOCR(use_angle_cls=True, lang='fr')

def Run_Paddle(Image_path):
    Start = time.time()
    result = paddle_reader.ocr(Image_path, det=True, rec=True)
    End = time.time()
    # Inference Time
    Time = End-Start
    # Process the result
    extracted_text = ''
    #print("Lenght Result: ",len(result))
    for line in result:
        if line is not None:
            for word in line:
                #print("Word: ",word)
                # Access the text part of the tuple
                extracted_text += word[1][0] + ' '  # Access the first element of the recognized text (the text itself)
            extracted_text += '\n'
    return extracted_text,Time
#############################################################################
############################## Docling v2 ###################################

from docling_core.types.doc import PictureItem
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions,EasyOcrOptions
from docling.document_converter import DocumentConverter,PdfFormatOption
import time

pipeline_options = PdfPipelineOptions()

pipeline_options.images_scale = 1
#pipeline_options.generate_page_images = True
#pipeline_options.generate_table_images = True
pipeline_options.do_ocr = True
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = True
ocr_options = EasyOcrOptions(force_full_page_ocr=True)
pipeline_options.ocr_options = ocr_options
pipeline_options.generate_picture_images = True

doc_converter = DocumentConverter(
    format_options={
        InputFormat.IMAGE: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

def Run_Docling(Image_path):
    # Convert the image to Markdown
    start_time = time.time()

    result = doc_converter.convert(Image_path)

    Markdown = result.document.export_to_markdown()
    # Itirate over all pictures in the document
    for element, _level in result.document.iterate_items():
        if isinstance(element, PictureItem):
            filename = "Docling.jpg"
            #element.image.pil_image.show()
            element.image.pil_image.save(filename)
            extracted_text,Time = Run_Paddle(filename)
            Markdown = extracted_text+"\n\n"+ Markdown
    end_time = time.time()
    Inference_time = end_time - start_time
    return Markdown,Inference_time

import pandas as pd
from tqdm import tqdm
import os

def Run_Docling_ALL(Folder_path):
    df = pd.DataFrame(columns=['InvoiceName', 'Text', 'Time(s)'])
    Files = os.listdir(Folder_path)
    Texts = []
    Times = []
    for file in tqdm(Files,desc="Docling Processing", unit="number"):
        print(f"{file} is processing")
        Invoice_path = os.path.join(Folder_path, file)
        Text_docling,Time_docling = Run_Docling(Invoice_path)
        Texts.append(Text_docling)
        Times.append(Time_docling)
    df['InvoiceName'] = Files
    df['Text'] = Texts
    df['Time(s)'] = Times
    # Saving the DataFrame to a CSV file
    df.to_csv(project_path+'CSVs/Docling_Results.csv', index=False)



download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:03<00:00, 1091.97it/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/latin_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer/latin_PP-OCRv3_rec_infer.tar


100%|██████████| 9930/9930 [00:00<00:00, 10571.20it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:00<00:00, 3044.90it/s]

[2024/11/24 19:42:00] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25,

#### Test

- Run on an Invoice

In [ ]:
Image_path = project_path+"Test_Invoices/2.jpg"
Markdown,Inference_time = Run_Docling(Image_path)
print(Inference_time)
print(Markdown)

- Run on All Invoices

In [ ]:
Run_Docling_ALL(project_path+"Test_Invoices")

In [8]:
import pandas as pd
Docling_Results = pd.read_csv(project_path+'CSVs/Docling_Results.csv')
Docling_Results.head()

,InvoiceName,Text,Time(s)
0,2.jpg,"KIPRINTSAEE RDC,Hay Habsani-Casablanca ICF0028...",11.785602
1,3.jpg,FUNART PLACE \n\n\nFUNSPLACE 295.AnleBAcimcn e...,10.819825
2,49.jpg,aaaialati \n\n\n\n\n\n\nalaix 1KT5 \n\n\nslinu...,14.915472
3,58.jpg,NETTO PAP PRODUITS DE NETTOYAGE ETD'HYGIENE \n...,11.136142
4,61.jpg,\n\n\n\n\n\n\n\n\n\n\n\nOBILIA DEEICE \n\n\n<!...,10.077137


## LLM

- From a quick search, you can find that LLM via API Providers like OpenAI, Anthropic... are the best in Information extraction.
But since we decide to keep everything localy, we need to test some of the local LLMs with Oallama and Huggingface.

- For All the tested models i use the q-4 quantized version of the model.

#### Install Dependencies

In [ ]:
!pip install langchain-openai==0.2.9

In [10]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

In [21]:
%xterm
# curl https://ollama.ai/install.sh | sh
# ollama serve & ollama pull llama3.2:3b
# Then Type Ctrl+C
# ollama pull llama3.2:3b

Launching Xterm...

<IPython.core.display.Javascript object>

#### LLM_Functions

In [13]:
################################################################################
################################ Structered Output ################################
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class Extraction_Facture(BaseModel):
    """Les Informations principales à extraire a partir d'une facture française"""
    Nom_facture: str = Field("", description="Le nom et le format(.jpg, .png, .jpeg) de l'image de la facture")
    Fournisseur: str = Field("", description="Le nom du fournisseur de la facture")
    date_facture: str = Field("", description="La date de la facture")
    num_facture: str = Field("", description="L'identifiant de la facture, peut être numérique ou alphanumérique")
    ttc: float = Field(0.0, description="Toutes Taxes Comprises. Il s'agit du prix final total payé par le client. Il inclut le prix Total HT et la Total TVA.")
    tt: float = Field(0.0, description="Le total hors taxes de la facture")
    tva: float = Field(0.0, description="Le Total taxes")



system = """
Vous êtes un assistant puissant dans l'extraction d'informations a partir des factures francaises. \n
- Un OCR est appliqué sur l'image de la facture, lire le text et extraire les informations suivantes :  \n

- Fournisseur, Date_Facture, Num_Facture, TTC, TT, TVA \n
- Les 3 prix TT,TTC,TVA doivent être numérique sans indication de devise. \n
- le nom de Fournisseur doit être en majuscule. \n
"""

## Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Le text donné par OCR ici :\n {OCR_Output} \n le Nom de facture est : \n {Nom_facture} \n"),
    ]
)

################################################################################
################################ LLM ###########################################
from langchain_openai import ChatOpenAI
import pandas as pd
import time

LLM = ChatOpenAI(
    model="llama3.2:3b",
    #temperature=0,
    api_key='ollama',
    base_url="http://localhost:11434/v1",
)
LLM_Structered = LLM.with_structured_output(Extraction_Facture)
LLM_Extractor = prompt | LLM_Structered

def Run_LLM(OCR_Output, Nom_facture):
    Input = {"OCR_Output":OCR_Output, "Nom_facture":Nom_facture}
    response = LLM_Extractor.invoke(Input)
    return response

# The Chain can raise an exception if the LLM fails to respect the structured output, so we can fix a retry number
RETRIES = 3

def Run_LLM_ALL(OCR_dataframe,pipelineName):
    LLM_Results = pd.DataFrame(columns=['Temps(s)','Nom_facture', 'Fournisseur', 'Date_Facture', 'Num_Facture', 'TTC', 'TT', 'TVA'])
    for index, row in OCR_dataframe.iterrows():
        print(f"Processing {index+1}/{len(OCR_dataframe)}...")
        for i in range(RETRIES):
            try:
                strat_time = time.time()
                response = Run_LLM(row['Text'], row['InvoiceName'])
                end_time = time.time()
                response_time = end_time - strat_time
                LLM_Results.loc[index] = [response_time,row['InvoiceName'], response.Fournisseur, response.date_facture, response.num_facture, response.ttc, response.tt, response.tva]
                #print(LLM_Results)
                break
            except Exception as e:
                #print(f"Error: {e}")
                LLM_Results.loc[index] = [0.0,row['InvoiceName'], '', '', '', 0.0, 0.0, 0.0]
                print(f"Retrying {i+1}/{RETRIES}...\n")

    # Saving the DataFrame to a CSV file
    LLM_Results.to_csv(project_path+f'CSVs/{pipelineName}_Results.csv', index=False)







#### Load OCR Results

In [22]:
import pandas as pd
df_docling = pd.read_csv(project_path+ 'CSVs/Docling_Results.csv')
df_docling.head()

,InvoiceName,Text,Time(s)
0,2.jpg,"KIPRINTSAEE RDC,Hay Habsani-Casablanca ICF0028...",11.785602
1,3.jpg,FUNART PLACE \n\n\nFUNSPLACE 295.AnleBAcimcn e...,10.819825
2,49.jpg,aaaialati \n\n\n\n\n\n\nalaix 1KT5 \n\n\nslinu...,14.915472
3,58.jpg,NETTO PAP PRODUITS DE NETTOYAGE ETD'HYGIENE \n...,11.136142
4,61.jpg,\n\n\n\n\n\n\n\n\n\n\n\nOBILIA DEEICE \n\n\n<!...,10.077137


In [23]:
index = 0
Text_docling = df_docling['Text'].values[index]
InvoiceName = df_docling['InvoiceName'].values[index]

#print("Text_docling :", Text_docling)
print("InvoiceName :", InvoiceName)

response = Run_LLM(Text_docling, InvoiceName)
response

InvoiceName : 2.jpg


Extraction_Facture(Nom_facture='2.jpg', Fournisseur='OUTIDIS SARL', date_facture='13/01/2023', num_facture='FV230007', ttc=630.6, tt=586.8, tva=44.0)

#### Run LLM on All Invoices

In [24]:
pipelineName = "llama3.2_3b_Docling"
Run_LLM_ALL(df_docling,pipelineName)
#pipelineName = "Qwen2.5_7b_Easy"
#Run_LLM_ALL(df_easyocr,pipelineName)

Processing 1/30...
Processing 2/30...
Processing 3/30...
Processing 4/30...
Processing 5/30...
Processing 6/30...
Processing 7/30...
Processing 8/30...
Processing 9/30...
Processing 10/30...
Processing 11/30...
Processing 12/30...
Processing 13/30...
Retrying 1/3...

Processing 14/30...
Processing 15/30...
Processing 16/30...
Retrying 1/3...

Processing 17/30...
Processing 18/30...
Processing 19/30...
Processing 20/30...
Processing 21/30...
Processing 22/30...
Retrying 1/3...

Retrying 2/3...

Processing 23/30...
Retrying 1/3...

Retrying 2/3...

Processing 24/30...
Processing 25/30...
Processing 26/30...
Processing 27/30...
Processing 28/30...
Processing 29/30...
Retrying 1/3...

Retrying 2/3...

Retrying 3/3...

Processing 30/30...


#### Evaluation Function

In [25]:
!pip install python-Levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 12.1 MB/s eta 0:00:00


In [26]:
import Levenshtein
import pandas as pd

def Calculate_Similarity(string1, string2):
    if pd.isna(string1):
        return 0
    distance = Levenshtein.distance(string1, string2)
    score = 1-(distance/max(len(string1), len(string2)))
    return score

def Evaluate(LLM_Results, Ground_Truth):
    Total_Similarity = 0
    Total_Margin = 0
    Total_Time = 0
    for index, Ground_row in Ground_Truth.iterrows():
        print("######## Invoice Number : ", index)
        Line_Similarity = 0
        Line_margin = 0

        # Strings Similarities
        Line_Similarity = Calculate_Similarity(LLM_Results.loc[index,'Fournisseur'], Ground_row['Fournisseur']) + Calculate_Similarity(LLM_Results.loc[index,'Date_Facture'], Ground_row['Date_Facture']) + Calculate_Similarity(LLM_Results.loc[index,'Num_Facture'], Ground_row['Num_Facture'])
        Total_Similarity += Line_Similarity/3
        print("Total Similarity : ", Total_Similarity)

        # Prices Margins
        Line_margin = abs(LLM_Results.loc[index,'TTC'] - Ground_row['TTC']) + abs(LLM_Results.loc[index,'TT'] - Ground_row['TT']) + abs(LLM_Results.loc[index,'TVA'] - Ground_row['TVA'])
        Total_Margin += Line_margin
        print("Total Margin : ", Total_Margin)

        Total_Time += LLM_Results.loc[index,'Temps(s)']

    Accuarcy = Total_Similarity/len(Ground_Truth)
    Average_Time = Total_Time/len(Ground_Truth)
    Average_Margin = Total_Margin/len(Ground_Truth)
    return Accuarcy, Average_Time, Average_Margin


#### Evaluation of Performance

In [27]:
LLM_Results = pd.read_csv(project_path+ f'CSVs/{pipelineName}_Results.csv')
Ground_Truth = pd.read_csv(project_path+'CSVs/grounding_truth.csv')
Accuarcy, Average_Time, Average_Margin = Evaluate(LLM_Results, Ground_Truth)

######## Invoice Number :  0
Total Similarity :  1.0
Total Margin :  200.60000000000002
######## Invoice Number :  1
Total Similarity :  1.7962962962962963
Total Margin :  620.6
######## Invoice Number :  2
Total Similarity :  2.7962962962962963
Total Margin :  1039.78
######## Invoice Number :  3
Total Similarity :  3.075281803542673
Total Margin :  4855.78
######## Invoice Number :  4
Total Similarity :  3.31231884057971
Total Margin :  5977.88
######## Invoice Number :  5
Total Similarity :  3.5789855072463768
Total Margin :  12662.48
######## Invoice Number :  6
Total Similarity :  3.8415127403618454
Total Margin :  14894.279999999999
######## Invoice Number :  7
Total Similarity :  4.061089459938565
Total Margin :  17334.399999999998
######## Invoice Number :  8
Total Similarity :  4.5498715112206165
Total Margin :  17564.39
######## Invoice Number :  9
Total Similarity :  5.34352230487141
Total Margin :  18404.39
######## Invoice Number :  10
Total Similarity :  5.615744527093632

In [28]:
print("Accuarcy :(%) ", Accuarcy*100)
print("Average_Time(s) : ", Average_Time)
print("Average_Margin (DH) : ", Average_Margin)

Accuarcy :(%)  41.82703043978585
Average_Time(s) :  1.663477643330892
Average_Margin (DH) :  330036245.534


In [ ]:
#Run this Only for the fist time to define the dataframe
#Results= pd.DataFrame(columns=['PipelineName','Accuarcy (%)', 'Average_Time (s)', 'Average_Margin (DH)'])
#Results.to_csv(project_path+'CSVs/Results.csv', index=False)

In [29]:
Results = pd.read_csv(project_path+'CSVs/Results.csv')
Results

,PipelineName,Accuarcy (%),Average_Time (s),Average_Margin (DH)
0,llama3.2_3b_Paddle,70.416943,1.561152,16072.474667


In [30]:
Results.loc[len(Results)] = [pipelineName,Accuarcy*100, Average_Time, Average_Margin]
Results.to_csv(project_path+'CSVs/Results.csv', index=False)

In [31]:
Results

,PipelineName,Accuarcy (%),Average_Time (s),Average_Margin (DH)
0,llama3.2_3b_Paddle,70.416943,1.561152,1.607247e+04
1,llama3.2_3b_Docling,41.827030,1.663478,3.300362e+08
